#  PROJET PYTHON POUR LA DATA SCIENCE :
## PAR : ELARNI Maroua & SADKI Ayoub  
***
## SCRAPING DU SITE AVITO.MA
***
***
***


## Importation des bibliothèques nécessaires :
### BeautifulSoup requests numpy et pandas 

In [7]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
from datetime import datetime , timedelta
import time

***
## Initialisation des variables :
***

In [8]:
nan = np.nan

#initialisation des compteurs de nombre d'annonces pour chaque catégorie qui va aussi servir pour key dans le dictionnaire par lequel on va créer les dataframes
nbr_ann , nbr_veh , nbr_imm , nbr_info , nbr_maison  , nbr_emp , nbr_hab , nbr_lois , nbr_entr , nbr_autr = 0,0,0,0,0,0,0,0,0,0

#initialisation des dataframe de chaque catégorie 
df_veh = pd.DataFrame(columns =['titre' , 'description','catégorie','prix', 'devise','ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien']) 
df_info = pd.DataFrame(columns =['titre','description','catégorie','prix' , 'devise', 'ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien']) 
df_maison=pd.DataFrame(columns =['titre','description','catégorie','prix','devise','ville','professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien']) 
df_imm = pd.DataFrame(columns =['titre','description','catégorie','prix','devise','ville', 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien']) 
df_emp = pd.DataFrame(columns =['titre','description','catégorie','prix','devise','ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien']) 
df_hab = pd.DataFrame(columns =['titre','description','catégorie','prix' , 'devise','ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien']) 
df_lois = pd.DataFrame(columns =['titre','description','catégorie','prix','devise','ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien']) 
df_entr = pd.DataFrame(columns =['titre','description','catégorie','prix','devise','ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien']) 
df_autr = pd.DataFrame(columns =['titre','description','catégorie','prix','devise','ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien'])

#fonction qui sépare un dictionnaire en deux listes de meme ordre keys et values 
def split_keys_values(d):
    keys = []
    values = []
    for k in d:
        keys.append(k)
        values.append(d[k])
    return ([keys , values])

***
## Prendre le lien a scraper (site avito):
https://www.avito.ma/fr/maroc/
***

In [19]:
#on entre (copier/coller) le lien du site dans la variable url de type str

################# """'''''' le lien le plus standard est : {{{{{{{{{{{{{{      'https://www.avito.ma/fr/maroc/'    }}}}}}}}}}}}}} ''''''''""" ######################

url = input("entrer le lien d'annonces que vous voulez scrapper (le lien doit être une page de recherche du site avito qui possède 35 annonces ) : ")

#on prend le codesource du site 
code = requests.get(url).text
soup = BeautifulSoup(code,'html.parser')

entrer le lien d'annonces que vous voulez scrapper (le lien doit être une page de recherche du site avito qui possède 35 annonces ) : https://www.avito.ma/fr/maroc/


***
## Code:
***

In [17]:
begin = time.time() #initialiser un timer
#entrer le nombre d'annonces qu'on veut scraper
n = int(input("entrer le nombre d'annonces que vous voulez scrapper (multiple de 35) : ")) 
while True : 
    if nbr_ann > n :
        break
    annonces = soup.find_all('h2' , {'class' : 'fs14'}) #liste des annoces de la page actuelle
    for annonce in annonces :
        link = annonce.find('a').get('href')   #lien de l'annonce en cours
        code_in = requests.get(link).text     #code source de l'annoce en cours
        soup_in = BeautifulSoup(code_in,'html.parser')   
        if soup_in.find('h1' , {'class' : 'page-header mbm'}) : #condition qui vérifie si l'annonce existe bel et bien (des fois le lien existe mais l'annonce est vide car le produit est vendu) 
            nbr_ann += 1 #incrémentation du compteur de nombre d'annonces totale
            cat = soup_in.find('ol',{'class' : 'breadcrumb'}).find_all('span')[-2].text.strip()  # catégorie de l'annonce en cours
            s_cat = soup_in.find('ol' , {'class' : 'breadcrumb'}).find_all('span')[-1].text.strip()  #sous catégorie de l'annonce en cours
            titre = soup_in.find('h1' , {'class' : 'page-header mbm'}).text.strip().split('\n')[0] # titre de l'annoce en cours
            vues = int(soup_in.find_all('div' , {'class' : 'panel-heading panel-heading-small'})[0].text.split()[1])  #nombre de vues de l'annonce en cours
            nom = soup_in.find('div' , {'class' : 'panel-footer'}).find('strong').text # nom de l'annonceur de l'annonce en cours
            date_heure = soup_in.find('abbr' , {'class' : 'date dtstart value'}).text.strip()  #date et heure de post de l'annonce en cours (séparé après dans Date et heure)
            if date_heure.split()[0] == "Aujourd'hui":
                Date = datetime.today().strftime('%d %b') 
            elif date_heure.split()[0] == 'Hier':
                Date = (datetime.today() - timedelta(days=1)).strftime('%d %b')
            else : 
                Date = ' '.join(date_heure.split()[0:2])

            heure = date_heure.split()[-1]
            prix_tag = soup_in.find('span' , {'class' : 'amount value'})   
            prix = float(prix_tag.text.replace(" ", "")) if prix_tag else nan #prix de l'annonce en cours
            devise_tag = soup_in.find('abbr' , {'class' : 'currency'})
            devise = devise_tag.text if devise_tag else nan  #devise de l'annonce en cours 
            pro_par_tag = soup_in.find('small' , {'class' : 'label'})
            pro_par = 'professionnel' if pro_par_tag else "particulier"  #type de l'annonce en cours (professionel ou bien particulier)                          
            desc_tag = soup_in.find('div' , {'class' : 'span10'}).find_all('p') 
            desc = ''
            for d in desc_tag:
                desc += d.text[1:]
            desc = desc.strip()   #desctiption de l'annonce en cours 
            loc = soup_in.find('h2' , {'class' : 'font-normal fs13 lh30 no-margin'}).text.strip() #localisation de l'annonce en cours
            
            #numéro de téléphone de l'annonce en cours
            if  'telephone: "0' in  code_in :
                index = code_in.index('telephone: "0')
                tel = 'MA: +212-' + code_in[index + 13 : index + 22]
            else:
                tel = np.nan   
                
            #informations supplémentaires prises de manière dynamique 
            #ces infos  n'existent pas dans toutes les annonces 
            #du coup si les infos existent on les prends et on les met dans un dictionnaire 'autres'
            #avec les keys sont les intitulés et les values sont l'information présentée
            autres_tag  = soup_in.find('aside').find('ul').find_all('h2')
            autres = {}
            for autre in autres_tag:
                if autre.find('strong') and autre.find('a'):
                    autres[autre.find('strong').text.strip()[:-1]] = autre.find('a').text.strip()

                elif autre.find('strong') :
                    if len(autre.text.split(':')[1].strip()) > 1 :
                        autres[autre.find('strong').text.strip()[:-1]] = autre.text.split(':')[1].strip()
                    else :
                        autres[autre.find('strong').text.strip()[:-1]] = nan 
        
#############################################################################################################
            #on check la catégorie de l'annonce en cours 
            #on traite les chaines de caractère on les rends int pour  les  colonnes nécessaires 
            #pour chaque catégorie on crée une dataframe composée d'une seule ligne (l'annonce en cours) qui sera appendé  à la dataframe initiale
            if s_cat in ['Engins Agricole' , 'Remorques Et Caravanes' , 'Engins Btp' , 'Camions' ]:
                cat = 'Véhicules'

            if cat.lower() == 'vehicules' : #or s_cat in ['Engins Agricole' , 'Remorques Et Caravanes' , 'Engins Btp' , 'Camions' ]:
                nbr_veh += 1

                if 'Année-Modèle' in autres and autres['Année-Modèle'] is not nan :
                    autres['Année-Modèle'] = int(autres['Année-Modèle'].split()[0])

                if 'Kilométrage' in autres and autres['Kilométrage'] is not nan : 
                    autres['Kilométrage'] = autres['Kilométrage'][-7:]
                    if autres['Kilométrage'][0] == '-':
                        autres['Kilométrage'] = int(autres['Kilométrage'][1:].strip().replace(' ',''))
                    else :
                        autres['Kilométrage'] = int(autres['Kilométrage'].strip().replace(' ' , '')) 

                k_v = split_keys_values(autres)

                d_veh = {}      
                d_veh[nbr_veh] = ([titre , desc , cat , prix , devise , loc , pro_par ,  vues , nom , tel , Date , heure ,link]+ k_v[1] )
                df_veh = df_veh.append(pd.DataFrame.from_dict(d_veh,  orient = 'index'  , columns = (['titre' , 'description' , 'catégorie' , 'prix' , 'devise' , 'ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien'] + k_v[0])))

            elif cat.lower() == 'immobilier' :   
                nbr_imm += 1
                if 'Nombre de pièces' in autres:
                    autres['Nombre de pièces']=int(autres['Nombre de pièces'])
                if 'Surface totale' in autres and type(autres['Surface totale']) != float : 
                    autres['Surface totale'] = float(autres['Surface totale'].split()[0]) 
                k_v = split_keys_values(autres)     
                d_imm = {}
                d_imm[nbr_imm] = ([titre , desc , cat ,   prix , devise , loc ,   pro_par ,  vues , nom , tel , Date , heure ,link] + k_v[1])                                                                                                                                                                                                                                      
                df_imm = df_imm.append(pd.DataFrame.from_dict(d_imm,  orient = 'index'  , columns = (['titre' , 'description' , 'catégorie' , 'prix' , 'devise' , 'ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien'] + k_v[0])))

            elif cat.lower() == 'informatique et multimedia' : 
                nbr_info += 1
                k_v = split_keys_values(autres)
                d_info = {}
                d_info[nbr_info] = ([titre , desc , cat , prix , devise , loc , pro_par ,  vues , nom , tel , Date , heure ,link] + k_v[1])  
                df_info = df_info.append(pd.DataFrame.from_dict(d_info,  orient = 'index'  , columns = (['titre' , 'description' , 'catégorie' , 'prix' , 'devise' , 'ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien'] + k_v[0])))

            elif cat.lower() == 'pour la maison et jardin' :
                nbr_maison += 1
                k_v = split_keys_values(autres)
                d_maison = {}
                d_maison[nbr_maison] = ([titre , desc , cat ,   prix , devise , loc ,   pro_par ,  vues , nom , tel , Date , heure ,link] + k_v[1])   
                df_maison = df_maison.append(pd.DataFrame.from_dict(d_maison,  orient = 'index'  , columns = (['titre' , 'description' , 'catégorie' , 'prix' , 'devise' , 'ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien'] + k_v[0])))

            elif cat.lower() == 'emploi et services' :
                nbr_emp += 1
                k_v = split_keys_values(autres)
                d_emp = {}
                d_emp[nbr_emp] = ([titre , desc , cat ,   prix , devise , loc ,   pro_par ,  vues , nom , tel , Date , heure ,link] + k_v[1])   
                df_emp = df_emp.append(pd.DataFrame.from_dict(d_emp,  orient = 'index'  , columns = (['titre' , 'description' , 'catégorie' , 'prix' , 'devise' , 'ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien'] + k_v[0])))

            elif cat.lower() == 'habillement et bien etre':
                nbr_hab += 1
                k_v = split_keys_values(autres)
                d_hab = {}
                d_hab[nbr_hab] = ([titre , desc , cat ,   prix , devise , loc ,   pro_par ,  vues , nom , tel , Date , heure ,link] + k_v[1])   
                df_hab = df_hab.append(pd.DataFrame.from_dict(d_hab,  orient = 'index'  , columns = (['titre' , 'description' , 'catégorie' , 'prix' , 'devise' , 'ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien'] + k_v[0])))

            elif cat.lower() == 'loisirs et divertissement':
                nbr_lois += 1
                k_v = split_keys_values(autres)
                d_lois = {}
                d_lois[nbr_lois] = ([titre , desc , cat ,   prix , devise , loc ,   pro_par ,  vues , nom , tel , Date , heure ,link] + k_v[1])   
                df_lois = df_lois.append(pd.DataFrame.from_dict(d_lois,  orient = 'index'  , columns = (['titre' , 'description' , 'catégorie' , 'prix' , 'devise' , 'ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien'] + k_v[0])))

            elif cat.lower() == 'entreprises': 
                nbr_entr += 1 
                k_v = split_keys_values(autres)
                d_entr = {}
                d_entr[nbr_entr] = ([titre , desc , cat ,   prix , devise , loc ,   pro_par ,  vues , nom , tel , Date , heure ,link] + k_v[1])   
                df_entr = df_entr.append(pd.DataFrame.from_dict(d_entr,  orient = 'index'  , columns = (['titre' , 'description' , 'catégorie' , 'prix' , 'devise' , 'ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien'] + k_v[0])))

            elif cat.lower() == 'autres':
                nbr_autr += 1
                k_v = split_keys_values(autres)
                d_autr = {}
                d_autr[nbr_autr] = ([titre , desc , cat ,   prix , devise , loc ,   pro_par ,  vues , nom , tel , Date , heure ,link] + k_v[1])  
                df_autr = df_autr.append(pd.DataFrame.from_dict(d_autr,  orient = 'index'  , columns = (['titre' , 'description' , 'catégorie' , 'prix' , 'devise' , 'ville' , 'professionnel/particulier' , 'vues' , 'nom' , 'telephone' , 'Date' , 'Heure', 'lien'] + k_v[0])))
    #le codesource  de la page suivante
    soup = BeautifulSoup(requests.get(soup.find('div' , {'class' : 'pagination text-center mt-2'}).find_all('li')[-1].find('a').get('href')).text,'html.parser')

end = time.time() #fin du timer
print('\nExecution time: ' + str(((end-begin))) + ' s\n')

entrer le nombre d'annonces que vous voulez scrapper (multiple de 35) : 1050

Execution time: 478.75840640068054 s



***
## ~ traitement de quelques colonnes : 
***

In [11]:
#convertion des colonnes Année-Modèle et Kilométrage en int car
#les colonnes Année-Modèle et Kilométrage dans la data frame de la catégorie véhicules était affiché sous forme de float 
#car nan était considéré comme tel par la dataframe et que la colonne devait etre du meme type 
#du coup ils étaient reconvertis en float a chaque fois

df_veh[['Année-Modèle']] = df_veh[['Année-Modèle']].astype('Int64')
df_veh[['Kilométrage']] = df_veh[['Kilométrage']].astype('Int64')

***
## Dataframe to csv: 
***

In [12]:

#concaténation des dataframe de toute les catégorie en une seule df_avito
pdList = [df_veh , df_imm ,  df_info , df_maison , df_lois , df_emp , df_hab , df_entr , df_autr]  
dfList = ['vehicules' , 'immobilier' ,  'informatique' , 'maison' , 'loisir' , 'emploi' , 'habillement' , 'entreprise' , 'autres']  
df_avito = pd.concat(pdList , ignore_index=True)

#création des fichier csv  a partir des dataframe pour chaque catégorie
df_veh.to_csv('vehi.csv' , index= False )
df_imm.to_csv('immo.csv' , index= False )
df_info.to_csv('infor.csv' , index= False )
df_lois.to_csv('loisi.csv' , index= False )
df_maison.to_csv('mais.csv' , index= False )
df_emp.to_csv('empl.csv' , index= False )
df_entr.to_csv('entre.csv' , index= False )
df_autr.to_csv('autre.csv' , index= False )
df_hab.to_csv('habi.csv' , index= False )
df_avito.to_csv('avito.csv' , index= False )